In [1]:
import os
import sys
import datetime

import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

from SRModels.deep_learning_models.ESRGAN_model import ESRGAN
from SRModels.loading_methods import load_dataset_as_patches
from SRModels.constants import ESRGAN_PATCH_SIZE, ESRGAN_STRIDE, RANDOM_SEED, ESRGAN_SCALE_FACTOR

# Enable memory growth (prevents full pre-allocation)
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    try:
        tf.config.experimental.set_memory_growth(gpu, True)
    except Exception as e:
        print(f"Memory growth not set: {e}")

c:\Users\bgmanuel\anaconda3\envs\py310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\bgmanuel\anaconda3\envs\py310\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.

In [2]:
HR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/HR"))
LR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/LR"))

In [3]:
X, Y = load_dataset_as_patches(HR_ROOT, LR_ROOT, mode="scale", patch_size=ESRGAN_PATCH_SIZE, stride=ESRGAN_STRIDE, scale_factor=ESRGAN_SCALE_FACTOR)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, shuffle=True, random_state=RANDOM_SEED)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, shuffle=True, random_state=RANDOM_SEED)

print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_val shape: {X_val.shape}, Y_val shape: {Y_val.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

X_train shape: (10497, 24, 24, 3), Y_train shape: (10497, 48, 48, 3)
X_val shape: (1167, 24, 24, 3), Y_val shape: (1167, 48, 48, 3)
X_test shape: (1296, 24, 24, 3), Y_test shape: (1296, 48, 48, 3)


In [4]:
BATCH_SIZE = 8     # Ajustar según memoria GPU
EPOCHS = 1

# Se define también el dataset de test (solo para evaluación posterior)
# Se normaliza a [-1,1] para que coincida con lo usado en entrenamiento
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(BATCH_SIZE)
test_dataset = test_dataset.map(lambda x,y: (x*2.0 - 1.0, y*2.0 - 1.0), num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

In [6]:
model = ESRGAN()

model.setup_model(
    scale_factor=2, 
    growth_channels=16, 
    num_rrdb_blocks=8, 
    input_shape=X_train.shape[1:],
    output_shape=Y_train.shape[1:],
    from_trained=False
)

GENERATOR SUMMARY
Model: "Generator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 lr_input (InputLayer)          [(None, 24, 24, 3)]  0           []                               
                                                                                                  
 initial_conv (Conv2D)          (None, 24, 24, 64)   1792        ['lr_input[0][0]']               
                                                                                                  
 rrdb_0_dense1_conv1 (Conv2D)   (None, 24, 24, 16)   9232        ['initial_conv[0][0]']           
                                                                                                  
 rrdb_0_dense1_concat1 (Concate  (None, 24, 24, 80)  0           ['initial_conv[0][0]',           
 nate)                                                            'rrdb_

In [7]:
# Clear any previous session
tf.keras.backend.clear_session()

# Train ESRGAN and capture callbacks for metrics
time_cb, mem_cb = model.fit(
    X_train=X_train, 
    Y_train=Y_train, 
    X_val=X_val, 
    Y_val=Y_val,
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE,
    use_augmentation=True, 
    use_mix=False,
    augment_validation=False
)

Training on GPU: ['/physical_device:GPU:0']
Epoch 1/1
  Step 10/1313 G_loss=52.3068 D_loss=1.3806 PSNR=18.99 SSIM=0.2348
  Step 20/1313 G_loss=62.7309 D_loss=1.3531 PSNR=21.15 SSIM=0.4082
  Step 30/1313 G_loss=38.4433 D_loss=1.3430 PSNR=20.99 SSIM=0.5195
  Step 40/1313 G_loss=51.3518 D_loss=1.3351 PSNR=19.91 SSIM=0.3916
  Step 50/1313 G_loss=48.0998 D_loss=1.3091 PSNR=20.81 SSIM=0.4443
  Step 60/1313 G_loss=72.2870 D_loss=1.3138 PSNR=18.41 SSIM=0.1983
  Step 70/1313 G_loss=40.8189 D_loss=1.2167 PSNR=21.56 SSIM=0.3704
  Step 80/1313 G_loss=49.1252 D_loss=1.1786 PSNR=18.93 SSIM=0.3504
  Step 90/1313 G_loss=34.4265 D_loss=1.1496 PSNR=22.10 SSIM=0.4720
  Step 100/1313 G_loss=50.1854 D_loss=1.2631 PSNR=19.52 SSIM=0.2526
  Step 110/1313 G_loss=46.9305 D_loss=1.1559 PSNR=20.41 SSIM=0.2367
  Step 120/1313 G_loss=37.1448 D_loss=1.1538 PSNR=22.01 SSIM=0.3773
  Step 130/1313 G_loss=44.2600 D_loss=1.1224 PSNR=19.96 SSIM=0.3412
  Step 140/1313 G_loss=48.7325 D_loss=1.0887 PSNR=19.04 SSIM=0.2403
  S

In [ ]:
results = model.evaluate(test_dataset)

metrics_dict = {
    "eval_pixel_loss": float(results["avg_pixel_loss"]),
    "eval_perceptual_loss": float(results["avg_perceptual_loss"]),
    "eval_psnr": float(results["avg_psnr"]),
    "eval_ssim": float(results["avg_ssim"]),
    "epoch_time_sec": time_cb.mean_time_value(),
    "memory": mem_cb.as_dict()
}

Evaluating model on test dataset...
Evaluation Results:
  Average PSNR: 25.5389
  Average SSIM: 0.7104
  Average Pixel Loss: 0.0911
  Average Perceptual Loss: 18.9737
{'eval_pixel_loss': 0.09109276979241843, 'eval_perceptual_loss': 18.973708550135296, 'eval_psnr': 25.53886792689194, 'eval_ssim': 0.7104075477447038, 'epoch_time_sec': 1694.4191871000003, 'memory': {'gpu_mean_current_mb': 268.8560791015625, 'gpu_peak_mb': 1539.365478515625}}


In [9]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Save the trained model
model.save(directory=f"models/ESRGAN/ESRGAN_{timestamp}", timestamp=timestamp)

Generator model saved to models/ESRGAN/ESRGAN_20250908_112228\ESRGAN_generator_x2_20250908_112228.h5
Discriminator model saved to models/ESRGAN/ESRGAN_20250908_112228\ESRGAN_discriminator_x2_20250908_112228.h5


In [10]:
# Save evaluation/time/memory metrics next to the model
metrics_path = os.path.abspath(os.path.join(os.getcwd(), f"models/ESRGAN/ESRGAN_{timestamp}/ESRGAN_{timestamp}_metrics.pkl"))

with open(metrics_path, "wb") as f:
    pickle.dump(metrics_dict, f)
    
print(f"Saved metrics to {metrics_path}")

Saved metrics to c:\Users\bgmanuel\InteligenciaArtificial\MasterInteligenciaArtificial\Periodo2\TFM\Super-Resolution-Images-for-3D-Printing-Defect-Detection\SRModels\deep_learning_models\models\ESRGAN\ESRGAN_20250908_112228\ESRGAN_20250908_112228_metrics.pkl
